# Preprocessing Mesh

In [9]:
def normalise(pcd):
    """
    Computes the normals of a point cloud. Normals are oriented 
    with respect to the input point cloud if normals exist.
    Next, converts float64 numpy array of shape (n, 3) to Open3D format.
    Computed points are returned
    """
    
    radius = 0.1
    max_nn = 300

    try:
        pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius,
                                                                                 max_nn),
                             fast_normal_computation=False)
        pcd.normals = o3d.utility.Vector3dVector(np.asarray(pcd.normals))
        return pcd
    except RuntimeError:
        print("Normals were not computed!")
        

def initialise_pointcloud(point_cloud):
    '''
    Addddd here
    '''
    pcd = o3d.geometry.PointCloud()
    normalised_pcd = normalise(pcd)
    pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
    
    return pcd


def visualise(pcd):
    '''
    Visualise Point Cloud
    '''
    o3d.visualization.draw_geometries([pcd])

    
def count_clusters(pcd):
    '''
    Use DBScan to generate clusters
    '''
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(pcd.cluster_dbscan(eps=100, min_points=100, print_progress=False))
    
    max_label = labels.max() + 1
    print(f"point cloud has {max_label} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
        
    return max_label


def outlier_removal_validity(no_of_clusters):
    if no_of_clusters == 0:
        print("Save the whole thing & dont remove outliers")
        return False
    else:
        print("check for outliers and remove")  
        return True
    

def remove_outliers(pcd):
    '''
    Remove outliers based on radius
    '''
    
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3.6 * avg_dist
    
    cl, ind = pcd.remove_radius_outlier(nb_points=32, radius=radius)
    
    return cl, ind


def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
    

def save_xyz(pcd, choice, filename):
    '''
    Save the xyz points
    '''
    o3d.io.write_point_cloud(output_path +
                             choice + 
                             filename + 
                             '.xyz',
                             pcd)

In [10]:
def preprocess_noise(choice):
    print("Working on noise files now...")
    
    for file in glob.glob(input_path + choice + "*.xyz"):
        point_cloud = np.loadtxt(file)
        filename = os.path.splitext(os.path.basename(file))[0]
        
        pcd = initialise_pointcloud(point_cloud)
        cl, ind = remove_outliers(pcd)
        save_xyz(cl, choice, filename)
        
        
def preprocess_mixed(choice):
    print("Working on mixed files now...")
    
    for file in glob.glob(input_path + choice + "*.xyz"):
        point_cloud = np.loadtxt(file)
        filename = os.path.splitext(os.path.basename(file))[0]
        
        pcd = initialise_pointcloud(point_cloud)
        cl, ind = remove_outliers(pcd)
        save_xyz(cl, choice, filename)

In [12]:
import numpy as np
import pandas as pd
import open3d as o3d
import os
import glob

import matplotlib.pyplot as plt

input_path = "../../data/ensemble/xyt/points/"
output_path = "../../data/ensemble/xyt/processed_points/"
data_choices = ["0/", "1/"]
data_types = ["noise", "mixed" ]

for choice, types in zip(data_choices, data_types):
    {'noise': preprocess_noise,
     'mixed': preprocess_mixed}[types](choice)

Working on noise files now...
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Fai

KeyboardInterrupt: 

# For Energy Regression

In [ ]:
input_path = "../../../data/energy/xyz/points/"
output_path = "../../../data/energy/xyz/processed_points/"

def save_xyz(pcd, filename):
    '''
    Save the xyz points
    '''
    o3d.io.write_point_cloud(output_path + 
                             filename + 
                             '.xyz',
                             pcd)
    
def preprocess_mixed():    
    for file in glob.glob(input_path + "*.xyz"):
        point_cloud = np.loadtxt(file)
        filename = os.path.splitext(os.path.basename(file))[0]
        pcd = initialise_pointcloud(point_cloud)
        cl, ind = remove_outliers(pcd)
        save_xyz(cl, filename)
        
preprocess_mixed()